Contact Clean Up Program

Tools:
    Table: Shows the % of contacts per record type that are using a field
    
    Function: threshold_per_type(p,n)
        => A table showing all the fields that are under p% usage for n number of records
        
    Function: contact_type_using_field(r,f)
        => A table showing all contacts of type r that are using field f
        
    Function: contacts_using_fields_in_criteria(p,n)
        => Gets the fields with in the given criteria. For each field, a table of the contacts using the field for each record type
        
    Function: date_range_of_contacts(t)
        => Given a table of contacts using a field, returns the range of the contacts 
    

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import altair as alt
import seaborn as sns
import datetime
import tables
alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

In [21]:
sf_contact_df = pd.read_csv('contacts.csv',  encoding = "cp1252")
sf_contact_df.to_pickle('contact.pkl')

c:\users\jmadubuko\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (6,7,8,11,12,13,14,15,23,24,27,30,31,32,33,34,35,37,38,39,40,41,52,55,56,57,58,64,66,67,68,70,73,74,75,76,77,80,81,83,85,87,91,92,95,96,97,98,100,101,103,105,107,108,109,110,112,114,115,116,117,118,119,120,121,122,123,124,125,127,131,134,135,136,137,138,139,140,141,142,143,144,145,146,152,158,167,170,172,174,175,176,177,178,179,181,182,183,184,185,186,187,189,190,191,194,195,196,197,201,212,217,223,224,225,226,227,230,232,233,236,239,250,251,252,261,262,263,264,265,268,269,270,274,282,292,295,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,316,317,318,319,320,321,323,325,326,327,328,329,331,333,338,339,341,342,347,348,350,352,353,354,356,357,359,360,364,375,377,394,404,413,414,415,417,418,424,426,434,437,438,439,440,441,453,455,456,458,459,462,463,464,465,466,470,471,472,473,475,476,478,479,480,481,482,483,484,485,487,489,490,491,495,4

In [28]:
sf_contact_df.to_hdf('contact.h5', key='df', mode='w')

c:\users\jmadubuko\appdata\local\programs\python\python36\lib\site-packages\pandas\core\generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->['ID', 'ACCOUNTID', 'LASTNAME', 'FIRSTNAME', 'SALUTATION', 'MIDDLENAME', 'SUFFIX', 'NAME', 'RECORDTYPEID', 'OTHERSTREET', 'OTHERCITY', 'OTHERSTATE', 'OTHERPOSTALCODE', 'OTHERCOUNTRY', 'MAILINGSTREET', 'MAILINGCITY', 'MAILINGSTATE', 'MAILINGPOSTALCODE', 'MAILINGCOUNTRY', 'MAILINGGEOCODEACCURACY', 'PHONE', 'FAX', 'MOBILEPHONE', 'HOMEPHONE', 'OTHERPHONE', 'ASSISTANTPHONE', 'REPORTSTOID', 'EMAIL', 'TITLE', 'DEPARTMENT', 'ASSISTANTNAME', 'LEADSOURCE', 'DESCRIPTION', 'CURRENCYISOCODE', 'OWNERID', 'CREATEDDATE', 'CREATEDBYID', 'LASTMODIFIEDDATE', 'LASTMODIFIEDBYID', 'SYSTEMMODSTAMP', 'LASTACTIVITYDATE', 'LASTVIEWEDDATE', 'LASTREFERENCEDDATE', 'EMAILBOUNCEDREASON', 'EMAILBOUNCEDDATE', 'PHOTOURL', 'ADDRESS_TYPE_

In [22]:
[i for i in list(sf_contact_df) if 'DATE' in i]

['CREATEDDATE',
 'LASTMODIFIEDDATE',
 'LASTACTIVITYDATE',
 'LASTCUREQUESTDATE',
 'LASTCUUPDATEDATE',
 'LASTVIEWEDDATE',
 'LASTREFERENCEDDATE',
 'EMAILBOUNCEDDATE',
 'DRAW_END_DATE__C',
 'END_DATE__C',
 'GOLDMINE_CREATED_DATE__C',
 'GOLDMINE_LAST_MODIFIED_DATE__C',
 'START_DATE__C',
 'CONVERSION_PROGRAM_DATE__C',
 'FIRST_APPOINTMENT_DATE__C',
 'BDACTIVEDATE__C',
 'APPIRIO_CORE__LAST_ACCESS_DATE__C',
 'WV_CARRIER_UPDATE_EMAILS__C',
 'AFMO_AGENCY_CARRIER_UPDATE_EMAILS__C',
 'CONTRACT_SIGNED_DATE__C',
 'DATE_COMPLIANCE_TRAINING_TAKEN__C',
 'DATE_LAPSED__C',
 'CANSELL_DATE__C',
 'DATE_LICENSE_EXPECTED__C',
 'MKTO71_ACQUISITION_DATE__C',
 'FIRST_APPLICATION_DATE__C',
 'CARRIER_UPDATE_SUB_ELIGIBLE_MARKETO__C',
 'NEWS_UPDATES_OPTIN_MARKETO__C',
 'DNC_EXPIRATION_DATE__C',
 'LAST_INTERESTING_MOMENT_DATE_MARKETO__C',
 'MKTO_SI__HIDEDATE__C',
 'MKTO_SI__LAST_INTERESTING_MOMENT_DATE__C',
 'LAST_CREATED_APPOINTMENT_DATE__C',
 'MARKETING_T_C_SENT_DATETIME__C',
 'UPDATE_DETAILS__C',
 'UPDATE_IMPACT_CU

In [4]:
sf_contact_df['LASTACTIVITYDATE'] = pd.to_datetime(sf_contact_df['LASTACTIVITYDATE'])
sf_contact_df.index = sf_contact_df['LASTACTIVITYDATE']

In [29]:
def fieldsByThreshold(df,up_percent, low_percent=0):
    #percentage of the fields usage in SF
    total_usage_per_col = df.count()/len(df);
    return total_usage_per_col[ total_usage_per_col.between(low_percent/100,up_percent/100)]

In [30]:
#dict of sf contacts by record type
sf_contact_by_type = {i : sf_contact_df[sf_contact_df['RECORD_TYPE_NAME__C'] == i] for i in sf_contact_df.RECORD_TYPE_NAME__C.unique()}

#fields usage per record type 
usage_map={i: fieldsByThreshold(sf_contact_by_type[i], 101) for i in sf_contact_df.RECORD_TYPE_NAME__C.unique()}
usage_df = pd.DataFrame(usage_map); 

In [31]:
def threshold_per_type(max_threshold, min_num_types): #input a percent, and a minimun number of records types to be under used by
    all_fields  = list(usage_df.index)
    
    fields = [i for i in all_fields if len( [ j for j in list(usage_df.loc[i] <= max_threshold/100) if j==True]) >= min_num_types]            
    return usage_df.loc[fields] #all the fields that fall under 'threshold' usage 'cnt' times
# example -> threshold_per_type(1,12)  all fields that have under 1% usage for 12 or more record types

In [32]:
def contact_type_using_field(record_type, field_name):
    if record_type =='All':
        contacts = sf_contact_df 
    else:
        contacts = sf_contact_by_type[record_type]
        
    return contacts[contacts[field_name].notnull()]

In [33]:
#get contacts using under used fields
def contacts_using_fields_in_criteria(thresh_percent, num_types):
    types = list(usage_df.columns)
    types.append('All')
    #fields that fit the criteria
    fields_in_threshold = list(threshold_per_type(thresh_percent, num_types).index)
    #dictionary containing each field in the criteria and the contacts using that field for each record type
    return {i:  {j: contact_type_using_field(j, i) for j in types} for i in fields_in_threshold}

In [34]:
def date_range_of_contacts(df):
    earliest = df.resample('M').count()[:1].index.date[0].isoformat()
    latest = df.resample('M').count()[-1:].index.date[0].isoformat()
    return f'from {earliest} to {latest}'

In [11]:
# usage_df[usage_df.apply(np.max, axis=1) < 0.1]  #all fields whos max usage is below 1%

In [12]:
list(usage_df.columns)

['Wholesaler Contact Record Type',
 'Business Contact Record Type',
 'Wealthvest Staff Record Type',
 'Contracted Producer Contact',
 'Producer Contact Record Type',
 'AFMO Contact Record Type',
 'Agency Contact Record Type',
 'Private Business Contact Record Type',
 'Private Operations Business Contact Record Type',
 'Vendor Contact Record Type',
 'FinancialForce Contact Record Type',
 'Public Business Contact Record Type']

In [14]:
contact_type_using_field('All','ADDITIONAL_FAX__C')

,ID,ISDELETED,MASTERRECORDID,ACCOUNTID,LASTNAME,FIRSTNAME,SALUTATION,MIDDLENAME,SUFFIX,NAME,...,ZZJEF_TEMP4__C,MARKETO_ACCESS__C,DEAL_SYSTEM_ACCESS__C,COMMISSION_SYSTEM_ACCESS__C,BIRTHDATE_MM_DD__C,TITLE__C,DEPARTMENT__C,MARKETING_EMAIL_ALIAS_KEY__C,MARKETO_PROFILE_PICTURE_URL__C,RECORD_STATUS__C
LASTACTIVITYDATE,,,,,,,,,,,,,,,,,,,,,
2018-08-09,0033600001WS7zQAAT,False,NaN,0013600001PHzpKAAT,Business,New,NaN,NaN,NaN,New Business,...,NaN,False,False,False,NaN,NaN,NaN,kfauls,NaN,"<img src=""/servlet/servlet.FileDownload?file= ..."


In [37]:
x= contacts_using_fields_in_criteria(0,12)

In [38]:
zero_usage_field = sorted(list(x.keys()))

In [1]:
from selenium import webdriver
browser = webdriver.Chrome()

In [2]:
browser.get('https://na30.salesforce.com/p/setup/layout/LayoutFieldList?type=Contact&setupid=ContactFields')

In [3]:
api_selector= '#CustomFieldRelatedList_body > table > tbody > tr.dataRow > td:nth-child(4)'
d_type_selector = '#CustomFieldRelatedList_body > table > tbody > tr.dataRow > td:nth-child(6)'
api_2_type_map = {i.text.upper(): j.text for i,j in zip(browser.find_elements_by_css_selector(api_selector),browser.find_elements_by_css_selector(d_type_selector))}


In [4]:
api_2_type_map

{'RECORD_STATUS__C': 'Formula (Text)',
 'X84_24_ASSISTANCE_NEEDED__C': 'Picklist',
 'A_P_ACCOUNT__C': 'Lookup(Account)',
 'ACCOUNT_ID__C': 'Formula (Text)',
 'ACCOUNT_ID_STRING__C': 'Text(100)',
 'ACCOUNT_TEXT__C': 'Text(100)',
 'MKTO71_ACQUISITION_DATE__C': 'Date/Time',
 'MKTO71_ACQUISITION_PROGRAM__C': 'Text(255)',
 'MKTO71_ACQUISITION_PROGRAM_ID__C': 'Number(18, 0)',
 'ACQUISITION_STAGE__C': 'Picklist',
 'PSE__ACTION_CALCULATE_UTILIZATION__C': 'Checkbox',
 'PSE__ACTION_UPDATE_CURRENT_TIME_PERIOD__C': 'Checkbox',
 'ACTIVE_STAFF__C': 'Formula (Checkbox)',
 'ACTIVE_TERRITORY__C': 'Formula (Text)',
 'ACTIVE_USER__C': 'Formula (Checkbox)',
 'ACTIVE_WHOLESALER__C': 'Formula (Checkbox)',
 'ACTIVE_WITH_GAINFULLY__C': 'Checkbox',
 'PSE__ACTUALS_LAST_UPDATE_DATE__C': 'Date/Time',
 'PSE__ACTUALS_LAST_UPDATED_BY__C': 'Lookup(User)',
 'ADDITIONAL_BANK_RULE_INDICATOR__C': 'Formula (Checkbox)',
 'ADDITIONAL_BANK_RULES__C': 'Long Text Area(32768)',
 'ADDITIONAL_CONTACTS__C': 'Lookup(Case)',
 'ADDIT

In [7]:
import json
with open('api-type map.json', 'w') as outfile:
    json.dump(api_2_type_map, outfile)

In [ ]:
non_formula_zero_usage_field = [i for i in zero_usage_field if 'Formula' not in api_2_type_map[i]]

In [9]:
zero_usage_fields = pd.read_json('0 usage fields.json')

In [15]:
zero_field_types = {i:j for i,j in api_2_type_map.items() if i in list(zero_usage_fields.to_dict()[0].values())}

In [37]:
pd.DataFrame({'API Name' : list(zero_field_types.keys()), 'Data Type': list(zero_field_types.values())}).to_csv('0% Usage Fields.csv', index=False)

In [39]:
pd.read_csv('0% Usage Fields.csv',)

,API Name,Data Type
0,ADDITIONAL_CONTACTS__C,Lookup(Case)
1,AFMO_AGENCY_CARRIER_UPDATE_EMAILS__C,Formula (Text)
2,AFMO_RECRUITER__C,Formula (Text)
3,ALLIANZ_APPROVED_IC_WHOLESALER__C,Picklist
4,ANNUITY_ABC__C,Picklist
5,APPROVED_MARKETING_MATERIAL_NOTES_AFMO__C,Long Text Area(32768)
6,APPROVED_SALES_CHANNELS_AFMO__C,Formula (Text)
7,AVERAGE_FIA_PRODUCTION__C,Picklist
8,BASE_SALESFORCE_PROFILE__C,Picklist
9,BEHAVIOR_SCORE_MARKETO__C,"Number(10, 0)"
